# 1. Install required library

In [ ]:
%pip install semantic-kernel

# 2. Load env variables

In [1]:
# load the environment variables file
from dotenv import load_dotenv
import os
import openai
from semantic_kernel.connectors.memory.azure_cosmosdb.utils import CosmosDBSimilarityType, CosmosDBVectorSearchType

# Environment variable obtained for Azure Cosmos DB for MongoDB vCore
AZCOSMOS_CONNSTR=os.getenv("AZCOSMOS_CONNSTR")
AZCOSMOS_API=os.getenv("AZCOSMOS_API")
AZCOSMOS_DATABASE_NAME="dbm"
AZCOSMOS_CONTAINER_NAME="dbm_saro"
AZCOSMOS_SEARCH_RESULTS = int(10)

# Envrionment variables obtained for Azure OpenAI
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_KEY") 
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
openai.api_embeddings_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
openai.api_version = "2023-07-01-preview"


# Vector search index parameters
index_name = "VectorSearchIndex"
vector_dimensions = (
    1536  # text-embedding-ada-002 uses a 1536-dimensional embedding vector
)
num_lists = 1
similarity = CosmosDBSimilarityType.COS  # cosine distance
kind = CosmosDBVectorSearchType.VECTOR_IVF 
m = 16 
ef_construction = 64
ef_search = 40

In [6]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)
from semantic_kernel.connectors.memory.azure_cosmosdb import (
    AzureCosmosDBMemoryStore,
)
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable


# Initialize the kernel
kernel = sk.Kernel()

# Load the chat deployment name, initialize the chat completions with the required parameters 
# Add the created chat service to the semantic kernel instance.
kernel.add_service(
    AzureChatCompletion(
        service_id="chat_completion",
        deployment_name=openai.api_deployment_name,
        endpoint=openai.api_base,
        api_key=openai.api_key,
    )
)

# Load the embeddings deployment name and initialize the text embedding with the required parameters, 
# Add the created embedding service to the semantic kernel instance.
kernel.add_service(
    AzureTextEmbedding(
        service_id="text_embedding",
        deployment_name=openai.api_embeddings_deployment_name,
        endpoint=openai.api_base,
        api_key=openai.api_key,
    )
)

store = await AzureCosmosDBMemoryStore.create(
            database_name=AZCOSMOS_DATABASE_NAME,
            collection_name=AZCOSMOS_CONTAINER_NAME,
            vector_dimensions=vector_dimensions,
            num_lists=num_lists,
            similarity=similarity,
            kind = kind, 
            m = m, 
            ef_construction = ef_construction,
            ef_search = ef_search,
            index_name=index_name,
            cosmos_connstr=AZCOSMOS_CONNSTR,
            cosmos_api=AZCOSMOS_API
        )

# Add the created memory store to the semantic kernel instance.
memory = SemanticTextMemory(storage=store, embeddings_generator=kernel.get_service("text_embedding"))


# each time it calls the embedding model to generate embeddings from your query
input = "What SARO is for education? Provide me the details"

result = await memory.search(AZCOSMOS_CONTAINER_NAME, input, AZCOSMOS_SEARCH_RESULTS, 0, False)
combined_result = ""
for item in result:
    combined_result += item.additional_metadata + ", "
        
prompt = """
    You are a chatbot that can have a conversations about any topic related to the provided context.
    You will always provide answers in this sample format:
    Saro Number:  SARO-ROVI-24-0006759
    Amount: 1,000,000
    Department: 21 - Department of Education 
    Agency: 002-Bureau of Elementary Education 
    Operating Unit: 210001 - National Capital Region 
    Purpose: Release of allotment for the construction of additional classrooms in selected public elementary schools in the National Capital Region.
    
    Give explicit answers from the provided context or say 'I don't know' if it does not have an answer.
    provided context: {{$db_record}}

    User: {{$input}}
    Chatbot:"""


execution_settings = sk_oai.OpenAITextPromptExecutionSettings(
   service_id="chat_completion",
    ai_model_id=openai.api_deployment_name,
    max_tokens=500,
    temperature=0.0,
    top_p=0.5
)

# Configure the prompt template with necessary input variables and execution settings
chat_prompt_template_config = sk.prompt_template.prompt_template_config.PromptTemplateConfig(
    template=prompt,
    name="grounded_response",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="db_record", description="The database records", is_required=True),
        InputVariable(name="input", description="The user input", is_required=True),
    ],
        execution_settings=execution_settings
)

# Create a chat function using the defined prompt and template configuration
chat_function = sk.functions.kernel_function_from_prompt.KernelFunctionFromPrompt(
    function_name="ChatGPTFunc2",
    plugin_name="chatGPTPlugin2",
    prompt=prompt, 
    prompt_template_config=chat_prompt_template_config
)

completions_result = await kernel.invoke(
    chat_function, 
    sk.functions.kernel_arguments.KernelArguments(input=input, db_record=combined_result)
)

print(completions_result)




There are several SAROs related to education. Here are some of them:

1. SARO Number: SARO-ROVI-24-0006804
   Amount: 1,800,000.00
   Department: 21 - Department of Education
   Agency: 047-Bureau of Alternative Learning System
   Operating Unit: 210009 - Caraga
   Purpose: Release of allotment for the implementation of alternative learning programs in Caraga.

2. SARO Number: SARO-ROVI-24-0006787
   Amount: 1,800,000.00
   Department: 21 - Department of Education
   Agency: 030-Bureau of Alternative Learning System
   Operating Unit: 210006 - Zamboanga Peninsula
   Purpose: Release of allotment for the implementation of alternative learning programs in Zamboanga Peninsula.

3. SARO Number: SARO-ROVI-24-0006769
   Amount: 2,300,000.00
   Department: 21 - Department of Education
   Agency: 012-Bureau of Alternative Learning System
   Operating Unit: 210003 - Northern Mindanao
   Purpose: Release of allotment for the implementation of alternative learning programs in Northern Mindanao.

